# Define Infrastructure Policy as a Constitution in Natural Language

## Introduction

In this notebook we will show you how to use Amazon Bedrock and Anthropic Claude for policy management and governance that helps organizations maintain control and compliance across their infrastructure and operations.


It will allow users to define policies in natural language without dependency on any domain-specific language (DSL). It integrates with terraform allowing organizations to define, implement, and enforce policies for infrastructure of their cloud and on-premises environments.


#### Use case

To demonstrate the natural language policy definition capabilities of Anthropic Claude via Amazon Bedrock, let's take the use case of creating IT security policies for a growing tech company.


#### Persona
You are Sarah, the newly appointed Chief Information Security Officer (CISO) at TechGrowth Inc., a rapidly expanding software company. As the company scales, you need to establish comprehensive IT security policies that align with industry standards and regulations. However, you're facing challenges:

1. Your team lacks expertise in policy writing.
2. You need policies that are clear and understandable to all employees, not just IT specialists.
3. The policies must be adaptable as the company grows and technology evolves.
4. You're under time pressure to implement these policies before the next board meeting.

#### Implementation
To fulfill this use case, in this notebook we will show how to define policies in plain natural language and using Foundation Model (FM) to validate if the terraform plan complies with the defined security guidelines.We will use the Anthropic Claude 3 Sonnet Foundation model using the Amazon Bedrock API and Langchain. 

#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore and langchain.

Notice `capture` command below, this will suppress the output of pip installation commands. This will take approx about 3 - 5 mins to complete. You will not see any output as we are suppressing the output using `capture` command. If you would like to see the ouput, please comment out the `capture` command and run the cell. In this case, ignore `Warnings` and `Errors` you may see.

In [36]:
%%capture
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain_aws --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [37]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [1]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
#from langchain_community.chat_models.bedrock import BedrockChat
from langchain_aws import ChatBedrock
from botocore.client import Config

In [2]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["policy", "terraform_code"],
    template="""/
    <policy>{policy}<policy>/
    <terraform>{terraform_code}</terraform>/
    You are a terraform code and plan analyzer Please verify the if the policies mentioned within <policy></policy> tags are satisfied in the the terraform script provided within <terraform></terraform> tags. /
    Please convey your decision as a yes or no and provide this response within <decision></decision> tags.
    Please add your analysis and explanation with <explanation></explanation> tags.
    Think step by step and provide a detailed explanation for your answer./
    """
    )

## Define Policies as a Constitution in Natural Language
Here, we define all our policies as a constitution. Each policy is equivalent of a directive principle in this constitution.

In [3]:
policy = """
1. all EC2 instances should have tags/
"""

## Terraform code

Below is the terraform code which will be evaluated to verify if it meets all the policy requirements

In [4]:
terraform_code = """terraform {/
  required_providers {/
    aws = {/
      source  = "hashicorp/aws"/
      version = "~> 4.16"/
    }/
  }/
  required_version = ">= 1.2.0"/
}/
provider "aws" {/
  region  = "us-west-2"/
  profile = "jack.roper"/
}/
resource "aws_instance" "example_server" {/
  ami           = "ami-04e914639d0cca79a"/
  instance_type = "t2.micro"/
  tags = {/
    Name = "JacksBlogExample"/
  }/
}"""

## Prompt template with policy and Terraform code as input

In [5]:
# format the prompt to add variable values
prompt_formatted_str: str = prompt.format(
    policy=policy,
    terraform_code=terraform_code)

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [6]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0' # change this to use a different version from the model provider


#llm = BedrockChat(model_id=modelId, client=bedrock_client)
llm = ChatBedrock(model_id=modelId)

## Model Invocation and Response Generation

Invoke the model and visualize the response

In [7]:
response =llm.invoke(prompt_formatted_str)
print(response.content)

<decision>Yes</decision>

<explanation>
Let's analyze the policy and the Terraform script step by step:

1. Policy requirement:
   The policy states that "all EC2 instances should have tags"

2. Terraform script analysis:
   a. The script defines an AWS provider and an EC2 instance resource.
   b. The EC2 instance is created using the resource "aws_instance" named "example_server".
   c. In the resource block, we can see a "tags" section:
      ```
      tags = {
        Name = "JacksBlogExample"
      }
      ```

3. Policy compliance:
   a. The EC2 instance defined in the Terraform script does have a tag.
   b. The tag is a key-value pair with the key "Name" and the value "JacksBlogExample".

4. Detailed explanation:
   - The policy requires that all EC2 instances should have tags.
   - In Terraform, tags are typically added to resources using the "tags" argument, which is exactly what we see in this script.
   - The script defines one EC2 instance, and this instance has a tag applie

## Conclusion
You have now experimented with how to analyze a terraform code to verify if it meets the policies defined in natural language

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You